@Diksha. This notebook will

- Generate some simulated click times and trial durations
- Generate some simulated choices from a model based on a set of generative parameters
- Maximize the likelihood of the parameters given the data
- Compute the Hessian of the ML parameters
- Verify that the generative parameters can be recovered

In [1]:
addprocs(48);

In [2]:
using PyPlot, StatsBase, JLD, Pandas, DataFrames, Distributions
@everywhere using choice_observation, latent_DDM_common_functions, helpers

In [56]:
#some constants
dt,n,map_str,ntrials = 1e-2,53,"exp",Int(5e4)

(0.01, 53, "exp", 2, 5.0, 50000)

In [57]:
#parameters of the latent model
pz = DataFrames.DataFrame(generative = vcat(1e-6,10.,-0.5,40.,1.,0.8,0.02), 
    name = vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    fit = vcat(falses(1),trues(4),trues(2)),
    initial = vcat(1.,5.,-5,100.,2.,0.2,0.08));

#initialize any parameter that is not going to be fit to its generative value
pz[:initial][pz[:fit] .== false] = pz[:generative][pz[:fit] .== false];

In [58]:
#parameters for the choice observation
pd = DataFrames.DataFrame(generative = 0.1, name = "bias", fit = true, initial = 0.);
#ammend to the latent parameter dataframe
pchoice = vcat(pz,pd);

In [59]:
#generate some simulated clicks times and trial durations
data = latent_DDM_common_functions.sample_clicks(ntrials,dt);

In [60]:
#simulate choices from the model given the generative parameters
choice_observation.sampled_dataset!(data,pchoice[:generative]; num_reps=1, rng=4);

In [61]:
#compute the likelihood of the data given the generative parameters
choice_observation.do_LL(pchoice[:generative],dt,data,n)

-11794.359448028483

In [62]:
#find the ML parameters
pchoice[:final],opt_output,state = choice_observation.do_optim(copy(pchoice[:initial]),
    pchoice[:fit],dt,data,n;map_str=map_str);

Iter     Function value   Gradient norm 
     0     2.732715e+04     5.231976e+03
     1     1.972505e+04     1.510253e+04
     2     1.506637e+04     3.192668e+04
     3     1.336507e+04     1.675301e+04
     4     1.260324e+04     2.423752e+03
     5     1.198576e+04     1.723058e+03
     6     1.197915e+04     1.210649e+03
     7     1.197543e+04     1.193942e+03
     8     1.193083e+04     3.243761e+03
     9     1.182682e+04     3.171932e+02
    10     1.181540e+04     9.849784e+02
    11     1.181090e+04     1.193125e+03
    12     1.180806e+04     1.054701e+03
    13     1.180415e+04     6.303073e+02
    14     1.180167e+04     4.182097e+02
    15     1.180049e+04     1.166656e+02
    16     1.179974e+04     4.552244e+01
    17     1.179944e+04     3.279070e+01
    18     1.179922e+04     4.457033e+01
    19     1.179912e+04     1.220111e+02
    20     1.179909e+04     1.368042e+02
    21     1.179909e+04     1.460241e+02
    22     1.179909e+04     1.446536e+02
    23     1.179

In [63]:
#compute the Hessian around the ML parameters
pchoice[:CI_z_plus], pchoice[:CI_z_minus], H = choice_observation.do_H(pchoice[:final],pchoice[:fit],
    dt,data,n;map_str=map_str)

#identify which ML parameters have generative parameters within the CI 
pchoice[:within_bounds] = (pchoice[:CI_z_minus] .< pchoice[:generative]) .& (pchoice[:CI_z_plus] .> pchoice[:generative]);

In [64]:
pchoice

,generative,name,fit,initial,final,CI_z_plus,CI_z_minus,within_bounds
1,1.0e-6,σ_i,false,1.0e-6,1.0e-6,1.0e-6,1.0e-6,false
2,10.0,B,true,5.0,10.0998,11.6094,8.82728,true
3,-0.5,λ,true,-5.0,-0.0870973,0.339384,-0.513566,true
4,40.0,σ_a,true,100.0,38.4873,49.5629,29.8867,true
5,1.0,σ_s,true,2.0,0.968198,1.3435,0.697733,true
6,0.8,ϕ,true,0.2,0.79014,0.862142,0.724151,true
7,0.02,τ_ϕ,true,0.08,0.0194703,0.0301317,0.0125812,true
8,0.1,bias,true,0.0,0.151051,0.306445,-0.00434367,true
